In [1]:
# %load retinanet_train.py
#!/usr/bin/env python

# In[1]:

print("Start importing package...")
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
import mymodel
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader
from mydataloader import MyDataset
import coco_eval
import csv_eval
import visdom
vis = visdom.Visdom(env='retinanet_seresnext101')

assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#参数
NAME="RSNA"
DATA_PATH = "/data/krf/dataset"
CSV_TRAINS = [DATA_PATH + "/csv_train0.csv",DATA_PATH + "/csv_train1.csv",DATA_PATH + "/csv_train2.csv",DATA_PATH + "/csv_train3.csv"]
CSV_VALS = [DATA_PATH + "/csv_val0.csv",DATA_PATH + "/csv_val1.csv",DATA_PATH + "/csv_val2.csv",DATA_PATH + "/csv_val3.csv"]
CSV_CLASSES = DATA_PATH + "/classes.csv"
DEPTH = 101
EPOCHS = 20
BATCH_SIZE=2
VAL_SIZE = 3000
#TRAIN_SIZE = 100
#数据预处理
import csv
import random
from tqdm import tqdm

Start importing package...
CUDA available: True


In [4]:
# with open(DATA_PATH+"/stage_1_train_labels.csv") as f:
#     reader = csv.reader(f)
#     rows=[row for row in  reader]
#     rows = rows[1:]
#     random.shuffle(rows)
#     for row in rows:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[1] == '' and row[2] == '' and row[3] == '' and row[4] == '':
#             row[5] = ''
#         else:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#     val_rows = rows[:VAL_SIZE]
#     train_rows = rows[VAL_SIZE:]
#     print(len(val_rows),len(train_rows))
#     with open(CSV_TRAIN,'w') as f2:
#         write = csv.writer(f2)
#         write.writerows(train_rows)
#         print("csv_train 写入完毕")
#     with open(CSV_VAL,'w') as f3:
#         write = csv.writer(f3)
#         write.writerows(val_rows)
#         print("csv_val 写入完毕")

# with open(CSV_CLASSES,'w') as f:
#     write = csv.writer(f)
#     row = ['1','0']
#     write.writerow(row)
#     print("csv_classes 写入完毕")



# #每次跑这个函数之前需要先删除之前的

# df = pd.read_csv(DATA_PATH+"/stage_1_train_labels.csv")
# train_images = os.listdir(DATA_PATH+"/stage_1_train_images")
# random.shuffle(train_images)#打乱图片顺序
# count = 0
# pos_cnt_train = [0,0,0,0]
# pos_cnt_val = [0,0,0,0]
# for img_name in tqdm(train_images):
#     results = df[df['patientId']==img_name.split('.')[0]].values
#     for row in results:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[5] == 1:
#             pos_cnt_val[count % 4] += 1
#         if row[1] >= 0 and row[1] <= 1024:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#         else:
#             row[1] = ''
#             row[2] = ''
#             row[3] = ''
#             row[4] = ''
#             row[5] = ''
#         with open(CSV_VALS[count % 4],'a') as f:
#             write = csv.writer(f)
#             write.writerow(row)
#         for i in range(4):
#             if count % 4 == i:
#                 continue
#             else:
#                 with open(CSV_TRAINS[count % 4],'a') as f:
#                     write = csv.writer(f)
#                     write.writerow(row) 
#     count += 1
# print(pos_cnt_train,pos_cnt_val)  

100%|██████████| 25684/25684 [01:19<00:00, 324.97it/s]

[0, 0, 0, 0] [2235, 2293, 2226, 2210]


In [2]:

print("Loading data...")

#%time
#制作数据loader
dataset_train = []
dataset_val = []

for i in range(4): 
    dataset_train.append(MyDataset(train_file=CSV_TRAINS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()])))
    dataset_val.append(MyDataset(train_file=CSV_VALS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Resizer()])))

#每次的sampler的参数：来源、batchsize、是否抛弃最后一层？？？

# num_workers 同时工作的组？collater:校验用的吧
dataloader_train = []
dataloader_val = []

for i in range(4):
    sampler = AspectRatioBasedSampler(dataset_train[i], batch_size=BATCH_SIZE, drop_last=False)
    sampler_val = AspectRatioBasedSampler(dataset_val[i], batch_size=1, drop_last=False)
    dataloader_train.append(DataLoader(dataset_train[i], num_workers=1, collate_fn=collater, batch_sampler=sampler))
    dataloader_val.append(DataLoader(dataset_val[i], num_workers=1, collate_fn=collater, batch_sampler=sampler_val))



Loading data...


In [3]:
retinanets = []
# Create the model
# if DEPTH == 18:
#     retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 34:
#     retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 50:
#     for i in range(4):
#         retinanets.append(model.resnet50(num_classes=dataset_train[i].num_classes(), pretrained=True))
# elif DEPTH == 101:
#     for i in range(4):
#         retinanets.append(model.resnet101(num_classes=dataset_train[i].num_classes(), pretrained=True)) 
# elif DEPTH == 152:
#     for i in range(4):
#         retinanets.append(model.resnet152(num_classes=dataset_train[i].num_classes(), pretrained=True))
# else:
#     raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')
#retinanet = torch.load('weights/RSNA_retinanet_5.pt')
for i in range(4):
    retinanets.append(mymodel.se_resnext101_32x4d(num_classes=dataset_train[i].num_classes()))
optimizer = []
scheduler = []
loss_hist = []
for i in range(4):
    retinanets[i] = retinanets[i].cuda()
    #变成并行
    retinanets[i] = torch.nn.DataParallel(retinanets[i]).cuda()
    #训练模式
    retinanets[i].training = True
    #学习率0.00001  
    optimizer.append(optim.Adam(retinanets[i].parameters(), lr=1e-5))
    #如果3个epoch损失没有减少则降低学习率
    scheduler.append(optim.lr_scheduler.ReduceLROnPlateau(optimizer[i], patience=2, verbose=True))
    # TODO 这是干什么 deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈：这里是定义了一个500的队列
    loss_hist.append(collections.deque(maxlen=1000))
    print('Num training images: {}'.format(len(dataset_train[i])))
# In[5]:



Num training images: 6421
Num training images: 6421
Num training images: 6421
Num training images: 6421


In [4]:
count = [0,0,0,0]
import traceback
for epoch_num in range(EPOCHS):

    for i in range(4):
        retinanets[i].train()
        retinanets[i].module.freeze_bn()
        epoch_loss = []
        
        for iter_num, data in enumerate(dataloader_train[i]):
            try:
                optimizer[i].zero_grad()

                classification_loss, regression_loss = retinanets[i]([Variable(data['img'].cuda().float()), Variable(data['annot'].cuda())])
                classification_loss = classification_loss.mean()
                regression_loss = regression_loss.mean()

                loss = classification_loss + regression_loss

                if bool(loss == 0):
                    continue
                #反向传播？
                loss.backward()

                #这是干嘛？？TODO
                torch.nn.utils.clip_grad_norm_(retinanets[i].parameters(), 0.1)

                #这?TODO
                optimizer[i].step()

                loss_hist[i].append(float(loss))

                epoch_loss.append(float(loss))

                print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist[i])))
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(loss_hist[i])]), win='train loss '+str(i), update='append' ,opts={'title':'train loss '+str(i)})
                count[i] += 1
                vis.save(['retinanet_seresnext101'])

                del classification_loss
                del regression_loss
            except Exception as e:
                print(e)
                continue


        print("Evaluating dataset")
    #     retinanet.eval()
    #     for index,data in enumerate(dataloader_val):
    #         #data = dataset[index]
    #         scale = data['scale']
    #         # run network
    #         #scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    #         print(Variable(data['img'].cuda()))
    #         scores, labels, boxes = retinanet(Variable(data['img'].cuda()))
    #         #mAP = csv_eval.evaluate(dataset_val,retinanet)
        try:
            mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
            vis.line(X=torch.Tensor([epoch_num]), Y=torch.Tensor([mAP[0][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
            vis.save(['retinanet_seresnext101'])
        except Exception as e:
            print(e)
            print( 'traceback.print_exc():', traceback.print_exc())
            continue
        #这一步也看不懂？？TODO 
        scheduler[i].step(np.mean(epoch_loss))

        torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
    #     torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(parser.dataset, epoch_num))

for i in range(4):
    retinanets[i].eval()
    torch.save(retinanets[i], 'weights_stage1/{}_model_final_seresnext101.pt'.format(i))




/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Epoch: 0 | Iteration: 2 | Classification loss: 0.56469 | Regression loss: 0.48521 | Running loss: 1.04990
Epoch: 0 | Iteration: 3 | Classification loss: 0.56461 | Regression loss: 0.53214 | Running loss: 1.07332
Epoch: 0 | Iteration: 4 | Classification loss: 0.56427 | Regression loss: 0.49650 | Running loss: 1.06914
Epoch: 0 | Iteration: 7 | Classification loss: 1.12773 | Regression loss: 1.06248 | Running loss: 1.34941
Epoch: 0 | Iteration: 8 | Classification loss: 0.56380 | Regression loss: 0.58180 | Running loss: 1.30865
Epoch: 0 | Iteration: 11 | Classification loss: 1.12723 | Regression loss: 1.09925 | Running loss: 1.46162
Epoch: 0 | Iteration: 12 | Classification loss: 1.12525 | Regression loss: 1.07197 | Running loss: 1.56670
Epoch: 0 | Iteration: 13 | Classification loss: 0.56309 | Regression loss: 0.55151 | Running loss: 1.51019
Epoch: 0 | Iteration: 14 | Classification loss: 0.56123 | Regression loss: 0.52122 | Running loss: 1.46266
Epoch: 0 | Iteration: 15 | Classification 

Epoch: 0 | Iteration: 202 | Classification loss: 0.72227 | Regression loss: 0.97027 | Running loss: 1.12805
Epoch: 0 | Iteration: 203 | Classification loss: 0.41907 | Regression loss: 0.49326 | Running loss: 1.12532
Epoch: 0 | Iteration: 204 | Classification loss: 0.35907 | Regression loss: 0.45819 | Running loss: 1.12147
Epoch: 0 | Iteration: 205 | Classification loss: 0.36677 | Regression loss: 0.51812 | Running loss: 1.11855
Epoch: 0 | Iteration: 206 | Classification loss: 0.27811 | Regression loss: 0.40129 | Running loss: 1.11319
Epoch: 0 | Iteration: 207 | Classification loss: 0.29537 | Regression loss: 0.45148 | Running loss: 1.10878
Epoch: 0 | Iteration: 208 | Classification loss: 0.35710 | Regression loss: 0.46477 | Running loss: 1.10536
Epoch: 0 | Iteration: 209 | Classification loss: 0.31710 | Regression loss: 0.48694 | Running loss: 1.10182
Epoch: 0 | Iteration: 216 | Classification loss: 0.34195 | Regression loss: 0.48927 | Running loss: 1.09867
Epoch: 0 | Iteration: 222 | 

Epoch: 0 | Iteration: 412 | Classification loss: 0.35486 | Regression loss: 0.44257 | Running loss: 1.03832
Epoch: 0 | Iteration: 414 | Classification loss: 0.31823 | Regression loss: 0.43963 | Running loss: 1.03651
Epoch: 0 | Iteration: 418 | Classification loss: 0.40337 | Regression loss: 0.48387 | Running loss: 1.03555
Epoch: 0 | Iteration: 419 | Classification loss: 0.37875 | Regression loss: 0.46550 | Running loss: 1.03433
Epoch: 0 | Iteration: 420 | Classification loss: 0.46582 | Regression loss: 0.52533 | Running loss: 1.03406
Epoch: 0 | Iteration: 425 | Classification loss: 0.33822 | Regression loss: 0.42810 | Running loss: 1.03237
Epoch: 0 | Iteration: 434 | Classification loss: 0.49378 | Regression loss: 0.49271 | Running loss: 1.03209
Epoch: 0 | Iteration: 435 | Classification loss: 0.34185 | Regression loss: 0.46733 | Running loss: 1.03070
Epoch: 0 | Iteration: 436 | Classification loss: 0.31849 | Regression loss: 0.45445 | Running loss: 1.02911
Epoch: 0 | Iteration: 437 | 

Epoch: 0 | Iteration: 629 | Classification loss: 0.34020 | Regression loss: 0.51464 | Running loss: 0.99736
Epoch: 0 | Iteration: 631 | Classification loss: 0.28340 | Regression loss: 0.51533 | Running loss: 0.99650
Epoch: 0 | Iteration: 634 | Classification loss: 0.36263 | Regression loss: 0.44149 | Running loss: 0.99567
Epoch: 0 | Iteration: 637 | Classification loss: 0.33044 | Regression loss: 0.55334 | Running loss: 0.99519
Epoch: 0 | Iteration: 643 | Classification loss: 0.30447 | Regression loss: 0.47118 | Running loss: 0.99425
Epoch: 0 | Iteration: 644 | Classification loss: 0.58005 | Regression loss: 0.47810 | Running loss: 0.99453
Epoch: 0 | Iteration: 645 | Classification loss: 0.32320 | Regression loss: 0.49470 | Running loss: 0.99378
Epoch: 0 | Iteration: 650 | Classification loss: 0.34573 | Regression loss: 0.28763 | Running loss: 0.99226
Epoch: 0 | Iteration: 651 | Classification loss: 0.25183 | Regression loss: 0.46542 | Running loss: 0.99110
Epoch: 0 | Iteration: 652 | 

Epoch: 0 | Iteration: 808 | Classification loss: 0.34208 | Regression loss: 0.45071 | Running loss: 0.95126
Epoch: 0 | Iteration: 811 | Classification loss: 0.22443 | Regression loss: 0.41399 | Running loss: 0.95024
Epoch: 0 | Iteration: 812 | Classification loss: 0.48497 | Regression loss: 0.89305 | Running loss: 0.95163
Epoch: 0 | Iteration: 814 | Classification loss: 0.49105 | Regression loss: 0.46178 | Running loss: 0.95163
Epoch: 0 | Iteration: 815 | Classification loss: 0.28963 | Regression loss: 0.59777 | Running loss: 0.95143
Epoch: 0 | Iteration: 821 | Classification loss: 0.36703 | Regression loss: 0.41208 | Running loss: 0.95087
Epoch: 0 | Iteration: 822 | Classification loss: 0.23069 | Regression loss: 0.50244 | Running loss: 0.95017
Epoch: 0 | Iteration: 826 | Classification loss: 0.20716 | Regression loss: 0.44195 | Running loss: 0.94921
Epoch: 0 | Iteration: 830 | Classification loss: 0.24900 | Regression loss: 0.52626 | Running loss: 0.94866
Epoch: 0 | Iteration: 831 | 

Epoch: 0 | Iteration: 1036 | Classification loss: 0.17638 | Regression loss: 0.44807 | Running loss: 0.92828
Epoch: 0 | Iteration: 1038 | Classification loss: 0.84904 | Regression loss: 0.96636 | Running loss: 0.93060
Epoch: 0 | Iteration: 1040 | Classification loss: 0.19834 | Regression loss: 0.45910 | Running loss: 0.92989
Epoch: 0 | Iteration: 1041 | Classification loss: 0.30273 | Regression loss: 0.51292 | Running loss: 0.92959
Epoch: 0 | Iteration: 1042 | Classification loss: 0.23482 | Regression loss: 0.50614 | Running loss: 0.92910
Epoch: 0 | Iteration: 1043 | Classification loss: 0.18692 | Regression loss: 0.42684 | Running loss: 0.92829
Epoch: 0 | Iteration: 1044 | Classification loss: 0.15901 | Regression loss: 0.47530 | Running loss: 0.92753
Epoch: 0 | Iteration: 1050 | Classification loss: 0.23179 | Regression loss: 0.54894 | Running loss: 0.92715
Epoch: 0 | Iteration: 1051 | Classification loss: 0.16844 | Regression loss: 0.44199 | Running loss: 0.92634
Epoch: 0 | Iteratio

Process Process-1:
Traceback (most recent call last):
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/data/krf/anaconda/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/data/krf/model/pytorch-retinanet/mydataloader.py", line 109, in __getitem__
    sample = self.transform(sample)
  File "/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/data/krf/model/

KeyboardInterrupt: 